### Importing 

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils as utils
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score, mean_absolute_error
from sklearn.metrics import f1_score
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

### Data Reading, Exploration, and Preprocessing

#### Data Loading and Initial Exploration
Loading the dataset from the 'lateness_data.json' file and examining its initial structure.

In [2]:
file_path = 'lateness_data.json'
data = pd.read_json(file_path)
data.head()

,direct_delivery,batched_pickup,transport_type,order_time,delivery_distance,order_preparation_time,responsible_id,store_latitude,store_longitude,client_latitude,client_longitude,status,status_time
0,yes,yes,automobile,2023-10-09 19:23:55,7798,10,4444,55.795518,37.631224,55.780525,37.700847,early,18
1,yes,yes,automobile,2023-07-31 11:43:13,553,10,3798,55.783786,37.624401,55.781943,37.628641,early,6
2,yes,yes,bicycle,2023-08-21 19:35:37,711,20,7595,55.729464,37.692976,55.732003,37.689528,early,9
3,yes,yes,automobile,2023-09-06 00:19:29,3538,10,3797,55.731702,37.581492,55.726069,37.604986,early,16
4,yes,yes,automobile,2023-09-06 19:23:28,4169,10,9509,55.781360,37.677339,55.787238,37.700311,early,5


#### Target Variable Transformation
Transforming the 'status' column to numerical values using label encoding.
For our classification task, it's essential to convert categorical labels (early, late, on time) into a numerical format to train the model effectively.

In [3]:
status_mapping = {
    'early': 2,
    'late': 0,
    'on time': 1
}

data['status'] = data['status'].map(status_mapping)

#### One-Hot Encoding for Categorical Features
Applying one-hot encoding to the 'direct_delivery' and 'batched_pickup' columns.
One-hot encoding is used to handle categorical variables, creating binary columns for each category. This is crucial for feeding categorical data into machine learning models.

In [4]:
data = pd.get_dummies(data, columns=['direct_delivery'], drop_first=True)
data = pd.get_dummies(data, columns=['batched_pickup'], drop_first=True)

#### Encode Transport Type
Create a mapping and use label encoding for the "transport_type" variable.
Similar to the target variable, label encoding is applied to the "transport_type" feature to convert categorical data into a numerical format for model training.

In [5]:
transport_type_mapping = {
    'foot': 0,
    'scooter': 1,
    'bicycle': 2,
    'automobile': 3
}
data['transport_type'] = data['transport_type'].map(transport_type_mapping)

#### Sort and Drop Time-related Columns
Sort the dataset based on the "order_time" column and drop it.
Sorting by time is essential in time-series data. Additionally, dropping the "order_time" column removes unnecessary temporal information for the current modeling task.

In [6]:
data.sort_values(by='order_time', inplace=True)
data.drop('order_time', axis=1, inplace=True)

#### Prepare Features and Targets
Create feature matrix X and target variables y for classification and regression tasks.
This step involves splitting the dataset into features (X) and target variables (y) required for training the multi-task model. The "status" variable is used for classification, and the "status_time" variable is used for regression.

In [7]:
X = data.drop(['status', 'status_time'], axis=1)
y_classification = data[['status']]
y_regression = data[['status_time']]

#### Split Data into Training and Validation Sets
Perform a train-test split to create training and validation sets for features and target variables in both classification and regression tasks.

This step is crucial for evaluating the model's performance on data it hasn't seen during training. It ensures that the model generalizes well to new data.

In [8]:
X_train, X_val, y_classification_train, y_classification_val, y_regression_train, y_regression_val = train_test_split(
    X, y_classification, y_regression, test_size=0.2, shuffle=False
)

#### Feature Scaling
Scaling selected numerical features using StandardScaler.
Scaling is necessary to bring numerical features to a similar scale, preventing certain features from dominating the learning process and ensuring effective model training.

In [9]:
columns_to_scale = ['delivery_distance', 'order_preparation_time', 'responsible_id', 'store_latitude', 'store_longitude', 'client_latitude', 'client_longitude']

scaler = StandardScaler()
X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])
X_val[columns_to_scale] = scaler.transform(X_val[columns_to_scale])

#### Feature Selection with Lasso Regression
Apply Lasso regression for feature selection in both classification and regression tasks.
Lasso regression helps identify and retain the most relevant features by penalizing the model for unnecessary ones. This step improves model interpretability and can enhance generalization.



For Classification Task

In [10]:
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_classification_train)
feature_importance_classification = lasso.coef_
important_dict_classification = dict(zip(X_train.columns, feature_importance_classification))

Select features with weights less than 0.01

In [11]:
feature_drop_classification = [feature_drop for feature_drop in important_dict_classification if important_dict_classification[feature_drop] < 0.01]

For Regression Task

In [12]:
lasso.fit(X_train, y_regression_train)
feature_importance_regression = lasso.coef_
important_dict_regression = dict(zip(X_train.columns, feature_importance_regression))

Select features with weights less than 0.01

In [13]:
feature_drop_regression = [feature_drop for feature_drop in important_dict_regression if important_dict_regression[feature_drop] < 0.01]

Identify features common to both tasks

In [14]:
feature_drop_both_task = [feature_drop for feature_drop in feature_drop_classification if feature_drop in feature_drop_regression]

Drop selected features from the training and validation sets

In [15]:
X_train.drop(feature_drop_both_task, axis=1, inplace=True)
X_val.drop(feature_drop_both_task, axis=1, inplace=True)

Check result

In [16]:
print(f"X_train.shape = {X_train.shape}")
print(f"X_val.shape = {X_val.shape}")
print(f"y_classification_train.shape = {y_classification_train.shape}")
print(f"y_classification_val.shape = {y_classification_val.shape}")
print(f"y_regression_train.shape = {y_regression_train.shape}")
print(f"y_regression_val.shape = {y_regression_val.shape}")

X_train.shape = (88610, 6)
X_val.shape = (22153, 6)
y_classification_train.shape = (88610, 1)
y_classification_val.shape = (22153, 1)
y_regression_train.shape = (88610, 1)
y_regression_val.shape = (22153, 1)


### Multi-Task Model Definition and Training

#### Multi-Task Model Definition
Define a multi-task neural network model with a simple architecture.
The model aims to perform both classification and regression tasks simultaneously. This architecture includes four hidden layers and two output layers for classification and regression, respectively.

In [17]:
class MultiTaskModel(nn.Module):
    def __init__(self, input_dim):
        super(MultiTaskModel, self).__init__()

        # Define hidden layers
        self.hidden_layer1 = nn.Linear(input_dim, 128)
        self.hidden_layer2 = nn.Linear(128, 64)
        self.hidden_layer3 = nn.Linear(64, 32)
        self.hidden_layer4 = nn.Linear(32, 16)

        # Output layers for classification and regression
        self.classification_output = nn.Linear(16, 3)
        self.regression_output = nn.Linear(16, 1)

    def forward(self, x):
        # Apply ReLU activation to hidden layers
        x = torch.relu(self.hidden_layer1(x))
        x = torch.relu(self.hidden_layer2(x))
        x = torch.relu(self.hidden_layer3(x))
        x = torch.relu(self.hidden_layer4(x))

        # For classification task use softmax activation function
        class_output = torch.softmax(self.classification_output(x), dim=1)

        return class_output, self.regression_output(x)


#### Data Transformation
Transform the data into a numeric format if it is not already in numeric form.
Machine learning models, especially deep learning models, require numeric input. This step ensures that the data is in the correct format for training.

In [18]:
# Convert data to float32
X_train_values = X_train.values.astype(np.float32)
X_val_values = X_val.values.astype(np.float32)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val_values, dtype=torch.float32)

# Convert classification target variables to long tensors
y_classification_train_tensor = torch.tensor(y_classification_train.values, dtype=torch.long).squeeze()
y_classification_val_tensor = torch.tensor(y_classification_val.values, dtype=torch.long).squeeze()

# Convert regression target variables to float32 tensors
y_regression_train_tensor = torch.tensor(y_regression_train.values, dtype=torch.float32).squeeze()
y_regression_val_tensor = torch.tensor(y_regression_val.values, dtype=torch.float32).squeeze()

#### Model, Loss Function, and Optimizer
Create the multi-task model and define the loss function and optimizer.

- **Model:** The architecture is designed as a multi-task model with shared hidden layers for both classification and regression tasks. This allows the model to extract hierarchical features that are useful for both tasks.
  
- **Loss Functions:**
  - **Classification Task:** CrossEntropyLoss is chosen for the classification task. This loss function is appropriate when dealing with multiple classes. The model predicts the probability distribution over the three classes (late, on time, early), and CrossEntropyLoss measures the difference between the predicted distribution and the true distribution.
  
  - **Regression Task:** L1Loss (Mean Absolute Error) is used for the regression task. L1Loss measures the absolute difference between the predicted and true values. It is suitable for tasks where we want the model to predict a numeric value (delivery time) with a level of tolerance.

**Optimizer:**  
Adam optimizer is selected for parameter updates during training. Adam is an adaptive learning rate optimization algorithm that combines the benefits of two other extensions of stochastic gradient descent, AdaGrad and RMSProp. It is well-suited for training deep neural networks and often converges faster than traditional stochastic gradient descent.

In [19]:
model = MultiTaskModel(input_dim=X_train.shape[1])

classification_criterion = nn.CrossEntropyLoss()  
regression_criterion = nn.L1Loss()  

optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Data Loading

Load the data into PyTorch DataLoader for both training and validation sets.

**DataLoader:** DataLoader provides features like batching, shuffling, and parallel data loading, enhancing the training speed and efficiency.

In [20]:
# Create DataLoader for training set
data_loader = DataLoader(utils.data.TensorDataset(X_train_tensor, y_classification_train_tensor, y_regression_train_tensor), batch_size=16, shuffle=False)

# Create DataLoader for validation set
data_val_loader = DataLoader(utils.data.TensorDataset(X_val_tensor, y_classification_val_tensor, y_regression_val_tensor), batch_size=16, shuffle=False)

#### TensorBoard Setup
Initialize TensorBoard for monitoring the model's performance during training.

In [21]:
# Set up TensorBoard for logging
# log_dir = './logs_task_1'
# writer = SummaryWriter(log_dir)

writer = SummaryWriter(comment='task_1.1_Model_first')

#### Model Training
Train the multi-task model using the defined architecture, loss functions, and optimization technique.

In [ ]:
# Model Training
num_epochs = 10
for epoch in range(num_epochs):

    all_labels_classification = []
    all_predictions_classification = []
    mae_regression_error = []
    loss_train = []

    print(f"epoch: {epoch + 1}")

    # train
    model.train()
    for batch_idx, (X_train_tensor_batched, target_classification, target_regression) in enumerate(data_loader):
        # Forward
        classification_output, regression_output = model(X_train_tensor_batched)

        # Calculate loss function for each task
        classification_loss = classification_criterion(classification_output, target_classification)
        regression_loss = regression_criterion(regression_output, target_regression)

        # Total loss function - sum loss functions for each tasks
        total_loss = classification_loss + regression_loss

        loss_train.append(total_loss.item())

        predicted_labels_train = torch.argmax(classification_output, dim=1)
        all_labels_classification.extend(target_classification.cpu().numpy())
        all_predictions_classification.extend(predicted_labels_train.cpu().numpy())

        mae_regression_error.append(mean_absolute_error(target_regression, regression_output.detach().numpy()))

        # Backward and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

    # Log metrics to Tensorboard
    writer.add_scalar('Train/Total Loss', sum(loss_train) / len(loss_train), epoch)
    writer.add_scalar('Train/Classification Accuracy', accuracy_score(all_labels_classification, all_predictions_classification), epoch)
    writer.add_scalar('Train/Regression MAE', sum(mae_regression_error) / len(mae_regression_error), epoch)
    writer.add_scalar('Train/Classification F1', f1_score(all_labels_classification, all_predictions_classification, average='weighted'), epoch)

    all_labels_classification = []
    all_predictions_classification = []
    mae_regression_error = []
    loss_val = []

    # val
    model.eval()
    for batch_idx, (X_val_tensor_batched, target_classification, target_regression) in enumerate(data_val_loader):
        # Predict
        classification_output, regression_output = model(X_val_tensor_batched)

        # Calculate loss function for each task
        classification_loss = classification_criterion(classification_output, target_classification)
        regression_loss = regression_criterion(regression_output, target_regression)

        # Total loss function - sum loss functions for each tasks
        total_loss = classification_loss + regression_loss

        loss_val.append(total_loss.item())

        predicted_labels_train = torch.argmax(classification_output, dim=1)
        all_labels_classification.extend(target_classification.cpu().numpy())
        all_predictions_classification.extend(predicted_labels_train.detach().numpy())

        mae_regression_error.append(mean_absolute_error(target_regression, regression_output.detach().numpy()))

    # Log metrics to Tensorboard
    writer.add_scalar('Val/Total Loss', sum(loss_train) / len(loss_train), epoch)
    writer.add_scalar('Val/Classification Accuracy', accuracy_score(all_labels_classification, all_predictions_classification), epoch)
    writer.add_scalar('Val/Regression MAE', sum(mae_regression_error) / len(mae_regression_error), epoch)
    writer.add_scalar('Val/Classification F1', f1_score(all_labels_classification, all_predictions_classification, average='weighted'), epoch)

#### Save model

In [23]:
# Save model
torch.save(model.state_dict(), 'multi_task_model.pth')

#### Load tensorboard

In [24]:
%load_ext tensorboard
%tensorboard --logdir runs

Launching TensorBoard...

#### Close Tensorboard writer

In [25]:
writer.close()

### Improved Multi-Task Model Definition, Training, and Hyperparameter Tuning

#### Classification Head Model
Define a separate model head for the classification task with a modified architecture.

**Task-Specific Architecture:** The classification head has a distinct architecture tailored for classifying whether an order will be delivered early, on time, or late.

In [26]:
# Classification Head Model
class ClassificationHead(nn.Module):
    def __init__(self, input_size):
        super(ClassificationHead, self).__init__()

        # Define hidden layers
        self.hidden_layer1 = nn.Linear(input_size, 128)
        self.hidden_layer2 = nn.Linear(128, 64)
        self.hidden_layer3 = nn.Linear(64, 32)
        self.hidden_layer4 = nn.Linear(32, 16)

        # Output layer for classification
        self.classification_output = nn.Linear(16, 3)

    def forward(self, x):
        # Forward pass through hidden layers with ReLU activation
        x = F.relu(self.hidden_layer1(x))
        x = F.relu(self.hidden_layer2(x))
        x = F.relu(self.hidden_layer3(x))
        x = F.relu(self.hidden_layer4(x))

        # Apply softmax activation for classification
        x = F.softmax(self.classification_output(x), dim=1)

        return x

#### Regression Head Model

Define a separate model head for the regression task with a modified architecture.

**Task-Specific Architecture:** The regression head has a distinct architecture tailored for predicting the difference between expected delivery times.

In [27]:
# Regression Head Model
class RegressionHead(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RegressionHead, self).__init__()

        # Define hidden layers
        self.hidden_layer1 = nn.Linear(input_size, 128)
        self.hidden_layer2 = nn.Linear(128, 64)
        self.hidden_layer3 = nn.Linear(64, 32)
        self.hidden_layer4 = nn.Linear(32, 16)

        # Output layer for regression
        self.regression_output = nn.Linear(16, output_size)

    def forward(self, x):
        # Forward pass through hidden layers with ReLU activation
        x = torch.relu(self.hidden_layer1(x))
        x = torch.relu(self.hidden_layer2(x))
        x = torch.relu(self.hidden_layer3(x))
        x = torch.relu(self.hidden_layer4(x))

        # Linear output for regression
        return self.regression_output(x)

#### Improved Model Architecture and Optimizers
Create separate model heads for classification and regression tasks with modified architectures. Also, define new loss functions and optimizers for each task.

- **Task-Specific Models:** Two separate model heads are defined, each tailored to its specific task - classification and regression.
- **Modified Architecture:** The architecture of the classification head includes a softmax activation function to produce probabilities for each class. The regression head, on the other hand, outputs a continuous value for the regression task.
- **Custom Loss Functions:** Different loss functions are selected for each task. CrossEntropyLoss is suitable for classification, while Mean Squared Error (MSE) Loss is chosen for regression.
- **Optimizers:** Separate optimizers are used for classification and regression tasks, allowing flexibility in adjusting learning rates independently.

In [28]:
# Classification Head Model
model_classification = ClassificationHead(input_size=X_train.shape[1])

# Regression Head Model
model_regression = RegressionHead(input_size=7, hidden_size=5, output_size=1)

# Loss functions for classification and regression
classification_criterion_improved = nn.CrossEntropyLoss()
regression_criterion_improved = nn.MSELoss()

# Optimizers for classification and regression
optimizer_classification_improved = optim.Adam(model_classification.parameters(), lr=0.001)
optimizer_regression_improved = optim.Adam(model_regression.parameters(), lr=0.001)

#### Create DataLoaders for train and validation data

In [29]:
data_loader_improved = DataLoader(utils.data.TensorDataset(X_train_tensor, y_classification_train_tensor, y_regression_train_tensor), batch_size=16, shuffle=False) 

In [30]:
data_val_loader_improved = DataLoader(utils.data.TensorDataset(X_val_tensor, y_classification_val_tensor, y_regression_val_tensor), batch_size=16, shuffle=False)

#### Training Epoch
Train the multi-task model for both classification and regression using an improved architecture. The training process involves forward passes for both tasks, calculation of loss functions, and optimization using task-specific optimizers.

In [31]:
def train_epoch(model_classification, model_regression, data_loader, optimizer_classification, optimizer_regression,
                classification_criterion, regression_criterion, writer, epoch):
    model_classification.train()
    model_regression.train()

    all_labels_classification_train = []
    all_predictions_classification_train = []
    mae_regression_error_train = []
    loss_train = []

    for batch_idx, (X_train_tensor_batched, target_classification, target_regression) in enumerate(data_loader):
        # Forward pass for classification
        classification_output_train = model_classification(X_train_tensor_batched)

        # Calculate classification loss
        classification_loss_train = classification_criterion(classification_output_train, target_classification)

        optimizer_classification.zero_grad()
        classification_loss_train.backward()
        optimizer_classification.step()

        # Detach intermediate value to avoid graph retention issues
        detached_classification_output_train = torch.argmax(classification_output_train, dim=1).detach().unsqueeze(1)

        # Forward pass for regression
        # Combine X_train_tensor_batched and classification_output_train
        combined_input_train = torch.cat((X_train_tensor_batched, detached_classification_output_train), dim=1)

        regression_output_train = model_regression(combined_input_train)

        # Calculate regression loss
        regression_loss_train = regression_criterion(regression_output_train, target_regression)

        optimizer_regression.zero_grad()
        regression_loss_train.backward()
        optimizer_regression.step()

        # Metrics calculation for training
        predicted_labels_train = torch.argmax(classification_output_train, dim=1)
        all_labels_classification_train.extend(target_classification.cpu().numpy())
        all_predictions_classification_train.extend(predicted_labels_train.cpu().numpy())

        mae_error_train = torch.abs(regression_output_train - target_regression).mean().item()
        mae_regression_error_train.append(mae_error_train)

        loss_train.append(classification_loss_train.item() + regression_loss_train.item())

    # Log metrics to Tensorboard for training
    writer.add_scalar('Train Improved/Total Loss', sum(loss_train) / len(loss_train), epoch)
    writer.add_scalar('Train Improved/Classification Accuracy', accuracy_score(all_labels_classification_train, all_predictions_classification_train), epoch)
    writer.add_scalar('Train Improved/Regression MAE', sum(mae_regression_error_train) / len(mae_regression_error_train), epoch)
    writer.add_scalar('Train Improved/Classification F1', f1_score(all_labels_classification_train, all_predictions_classification_train, average='weighted'), epoch)


#### Evaluation Epoch Explanation
Perform the evaluation of the multi-task model on the validation set for both classification and regression tasks.

In [32]:
def eval_epoch(model_classification, model_regression, data_loader, classification_criterion, regression_criterion, writer, epoch):
    # Set models to evaluation mode
    model_classification.eval()
    model_regression.eval()

    # Initialize lists to store evaluation metrics
    all_labels_classification_eval = []
    all_predictions_classification_eval = []
    mae_regression_error_eval = []
    loss_eval = []

    # Disable gradient computation during evaluation
    with torch.no_grad():
        for batch_idx, (X_eval_tensor_batched, target_classification_eval, target_regression_eval) in enumerate(data_loader):
            # Forward pass for classification
            classification_output_eval = model_classification(X_eval_tensor_batched)

            # Calculate classification loss
            classification_loss_eval = classification_criterion(classification_output_eval, target_classification_eval)

            # Detach intermediate value to avoid graph retention issues
            detached_classification_output_eval = torch.argmax(classification_output_eval, dim=1).detach().unsqueeze(1)

            # Forward pass for regression
            # Combine X_eval_tensor_batched and classification_output_eval
            combined_input_eval = torch.cat((X_eval_tensor_batched, detached_classification_output_eval), dim=1)

            regression_output_eval = model_regression(combined_input_eval)

            # Calculate regression loss
            regression_loss_eval = regression_criterion(regression_output_eval, target_regression_eval)

            # Metrics calculation for evaluation
            predicted_labels_eval = torch.argmax(classification_output_eval, dim=1)
            all_labels_classification_eval.extend(target_classification_eval.cpu().numpy())
            all_predictions_classification_eval.extend(predicted_labels_eval.cpu().numpy())

            mae_error_eval = torch.abs(regression_output_eval - target_regression_eval).mean().item()
            mae_regression_error_eval.append(mae_error_eval)

            loss_eval.append(classification_loss_eval.item() + regression_loss_eval.item())

    # Metrics calculation for evaluation
    eval_accuracy = accuracy_score(all_labels_classification_eval, all_predictions_classification_eval)
    eval_mae = sum(mae_regression_error_eval) / len(mae_regression_error_eval)
    eval_loss = sum(loss_eval) / len(loss_eval)

    # Log metrics to Tensorboard for evaluation
    writer.add_scalar('Eval Improved/Total Loss', eval_loss, epoch)
    writer.add_scalar('Eval Improved/Classification Accuracy', eval_accuracy, epoch)
    writer.add_scalar('Eval Improved/Regression MAE', eval_mae, epoch)
    writer.add_scalar('Eval Improved/Classification F1', f1_score(all_labels_classification_eval, all_predictions_classification_eval, average='weighted'), epoch)


#### TensorBoard Setup
Initialize TensorBoard for monitoring the model's performance during training.

In [33]:
# Set up TensorBoard for logging
# log_dir = './logs_task_1'
# writer = SummaryWriter(log_dir)

writer = SummaryWriter(comment='task_1.2_Model_second')

#### Model Training and Evaluation
Train and evaluate the improved multi-task model for a specified number of epochs.

Training the model involves optimizing its parameters using the training dataset, while evaluation helps assess its performance on the validation set. Iterating through epochs allows the model to learn and adjust its weights over multiple passes through the training data, enhancing its ability to make accurate predictions.

In [34]:
# Model Training and Evaluation
num_epochs = 10

for epoch in range(num_epochs):
    print(f"epoch = {epoch + 1}")

    # Training
    train_epoch(model_classification, model_regression, data_loader_improved,
                optimizer_classification_improved, optimizer_regression_improved,
                classification_criterion_improved, regression_criterion_improved, writer, epoch)

    # Evaluation
    eval_epoch(model_classification, model_regression, data_val_loader_improved,
               classification_criterion_improved, regression_criterion_improved, writer, epoch)

epoch = 1


C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorre

epoch = 2


C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect

epoch = 3


...

C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect

epoch = 9


C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect

epoch = 10


C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\progr\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect

#### Save trained model parameters

In [35]:
torch.save(model_classification.state_dict(), 'model_classification.pth')
torch.save(model_regression.state_dict(), 'model_regression.pth')

#### Reload tensorboard

In [36]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

#### Close writer

In [37]:
writer.close()